### I. Access Data

In [2]:
storage_account_name = "cobaltstor"
storage_account_access_key = "jHVua****"

file_location = "wasbs://source@cobaltstor.blob.core.windows.net/kyphosis.csv"
file_type = "csv"

spark.conf.set("fs.azure.account.key."+storage_account_name+".blob.core.windows.net", storage_account_access_key)

In [3]:
df = spark.read.format(file_type).option("inferSchema", "true").option("header","true").load(file_location)

In [4]:
df_pd = df.toPandas()
df_pd.describe()

Out[ 25 ]: 
 Age Number Start
count 81.000000 81.000000 81.000000
mean 83.654321 4.049383 11.493827
std 58.104251 1.619423 4.883962
min 1.000000 2.000000 1.000000
25% 26.000000 3.000000 9.000000
50% 87.000000 4.000000 13.000000
75% 130.000000 5.000000 16.000000
max 206.000000 10.000000 18.000000

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df_pd.head()

Out[ 11 ]: 
 Kyphosis Age Number Start
0 absent 71 3 5
1 absent 158 3 14
2 present 128 4 5
3 absent 2 5 1
4 absent 1 4 15

### II. EDA

In [8]:
gg = sns.pairplot(df_pd,hue='Kyphosis',palette='Set1')
display(gg.fig)

In [9]:
sns.set(style="white")

g = sns.PairGrid(df_pd, diag_sharey=False)
g.map_lower(sns.kdeplot)
g.map_diag(sns.kdeplot, lw=3)

g.map_upper(sns.regplot)

display(g.fig)


### III. Train Test Split

In [11]:
from sklearn.model_selection import train_test_split
X = df_pd.drop('Kyphosis',axis=1)
y = df_pd['Kyphosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

### IV.1.1. Modeling : Decision Trees

In [13]:
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(random_state=13)
dtree.fit(X_train,y_train)

Out[ 51 ]: 
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
 max_features=None, max_leaf_nodes=None,
 min_impurity_split=1e-07, min_samples_leaf=1,
 min_samples_split=2, min_weight_fraction_leaf=0.0,
 presort=False, random_state=13, splitter='best')

### IV.1.2. Prediction and Evaluation

In [15]:
predictions = dtree.predict(X_test)

In [16]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_test,predictions))

precision recall f1-score support

 absent 0.71 0.88 0.79 17
 present 0.50 0.25 0.33 8

avg / total 0.65 0.68 0.64 25

In [17]:
print(confusion_matrix(y_test,predictions))

[[15 2]
 [ 6 2]]

### IV.2.1. Modeling : Random Forests

In [19]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)

Out[ 57 ]: 
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
 max_depth=None, max_features='auto', max_leaf_nodes=None,
 min_impurity_split=1e-07, min_samples_leaf=1,
 min_samples_split=2, min_weight_fraction_leaf=0.0,
 n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
 verbose=0, warm_start=False)

### IV.2.2. Prediction and Evaluation

In [21]:
rfc_pred = rfc.predict(X_test)

In [22]:
print(confusion_matrix(y_test,rfc_pred))

[[17 0]
 [ 6 2]]

In [23]:
print(classification_report(y_test,rfc_pred))

precision recall f1-score support

 absent 0.74 1.00 0.85 17
 present 1.00 0.25 0.40 8

avg / total 0.82 0.76 0.71 25